In [1]:
using Pkg; Pkg.activate("."); 
using MUST
using PyPlot

MUST.@import_dispatch "/u/peitner/DISPATCH/dispatch2" ;
#MUST.@import_dispatch "../../../dispatch2_clean/dispatch2" ;

  Activating environment at `/usr/local/autofs.shared/shared-storage/eitner/model_grid/MUST.jl/examples/dispatch2multi/Project.toml`


ErrorException: /u/peitner/DISPATCH/dispatch2/utilities/python/ is not a dispatch installation.

In [2]:
input_name  = "TSO_sun_Magg_rturc"
output_name = "TSO_sun_Magg"
eos_name    = "TSO_sun_Magg_v1"


input_name  = "TAB_sun_profile_hRes"
output_name = "TSO_sun_hRes"
eos_name    = "solar_lte";


input_name  = "TSO_sun_TDZv2_v3.1"
output_name = "TSO_sun_TDZv2_v3.1"
eos_name    = "TSO_sun_Magg_v3.1";

In [3]:
model_box = MUST.Boxes(MUST.@in_dispatch("data/$(input_name)"))
m = model_box[1][end-1];

BoundsError: BoundsError: attempt to access 0-element Vector{Any} at index [-1]

In [4]:
eos = MUST.SquareGasEOS(MUST.@in_dispatch("input_data/$(eos_name)"));

SystemError: SystemError: opening file "/u/peitner/DISPATCH/dispatch2/experiments/stellar_atmospheres/input_data/TSO_sun_Magg_v3.1/tabparam.in": No such file or directory

In [5]:
MUST.multiBox(m, eos, output_name, downsample_xy=20)

UndefVarError: UndefVarError: m not defined

In [6]:
b_d = MUST.multiBox(output_name);
@show size(b_d);

SystemError: SystemError: opening file "TSO_sun_TDZv2_v3.1.mesh": No such file or directory

In [7]:
b_m = MUST.multiBox("/u/peitner/MULTI3D/Multi3D/input_multi3d/atmos/t5777g44m0005_20.5x5x230");

SystemError: SystemError: opening file "/u/peitner/MULTI3D/Multi3D/input_multi3d/atmos/t5777g44m0005_20.5x5x230.mesh": No such file or directory

In [8]:
t = MUST.plane_statistic(MUST.mean, b_d, :T)
d = MUST.plane_statistic(MUST.mean, b_d, :d)

t_m = MUST.plane_statistic(MUST.mean, b_m, :T)
d_m = MUST.plane_statistic(MUST.mean, b_m, :d)

plt.scatter(log10.(b_d[:d]), b_d[:T], s=0.5, alpha=0.2, label="Dispatch", color="blue")
plt.scatter(log10.(b_m[:d]), b_m[:T], s=5.0, alpha=0.2, label="Stagger", color="red")

plt.plot(log10.(d), t, color="k", lw=3)
plt.plot(log10.(d_m), t_m, color="0.5", lw=3)

#plt.xlim(log(10, 1.e-7), log(10, 1e-6))
#plt.ylim(8000,14000)
plt.legend()

UndefVarError: UndefVarError: b_d not defined

In [9]:
plt.scatter(b_d[:T], b_d[:uz], s=0.5, alpha=0.2, color="blue", label="Dispatch")
plt.scatter(b_m[:T], b_m[:uz], s=5.0, alpha=0.2, color="red", label="Stagger")

plt.legend();

UndefVarError: UndefVarError: b_d not defined

In [10]:
plt.scatter(b_d[:T], log10.(b_d[:Ne]), s=0.5, alpha=0.2, color="blue", label="Dispatch")
plt.scatter(b_m[:T], log10.(b_m[:Ne]), s=5.0, alpha=0.2, color="red", label="Stagger")

plt.legend();

UndefVarError: UndefVarError: b_d not defined

In [11]:
mask_m = b_m[:T] .< 6000.0
mask_d = b_d[:T] .< 6000.0

u_m  = b_m[:uz][mask_m] ./ 1e5
u_d  = b_d[:uz][mask_d] ./ 1e5

plt.hist(reshape(u_m, :), bins=20, alpha=0.2, color="red", density=true, label="Stagger")
plt.hist(reshape(u_d, :), bins=100, alpha=0.2, color="blue", density=true, label="Dispatch")

plt.title("Uz distribution towards the surface")
plt.xlabel("Uz [km/s]")

plt.legend();

UndefVarError: UndefVarError: b_m not defined

In [12]:
plt.scatter(b_d[:T], log.(10, b_d[:Ne]))

UndefVarError: UndefVarError: b_d not defined